In [1]:
using Revise

In [2]:
using Graphs, OffsetArrays


In [3]:
using LammpsAnalysis

[ Info: Precompiling LammpsAnalysis [f54fdea9-a25b-4801-be39-89e20f64ecdd]


In [4]:
BasePath="/localscratch/HPS_DATA/HPS-Alpha/SLAB/SLAB_Restart_Calvados2/"
Protein="RS41_RRM"
Temp=280
prot= LammpsAnalysis.initData(BasePath*"$(Protein)/$(Temp)K/", "$(Protein)_slab.lmp";Reparse=false)  

Cannot find field: Dict{String, Any}() with key LammpsVariables.
Cannot find field: 1//1 with key Reduce.
Cannot find field: Dict{String, Int32}("pe" => 6, "etotal" => 5, "ecoul" => 9, "edihed" => 13, "time" => 2, "evdwl" => 8, "ke" => 7, "step" => 1, "press" => 4, "eangle" => 12, "epair" => 10, "ebond" => 11, "temp" => 3) with key EnergyDict.
Cannot find field: Float32[] with key SlabHistogramSeries.
Cannot find field: Matrix{Float32}(undef, 0, 0) with key Energies.
Cannot find field: 100000 with key EnergyWriteOutFreq.
Cannot find field: 0 with key NReplica.
Cannot find field: 100000 with key TrajWriteOutFreq.


LammpsAnalysis.SimData{Float32, Int32}("RS41_RRM_slab", 1, 4402, 33400, 33200, 33000, 32800, 200, 22, 1, 1, 146, 280.0f0, 2, Float32[-135.0 135.0; -750.0 750.0; -135.0 135.0], Float32[270.0, 1500.0, 270.0], ["M',KPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVK',", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVK", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVK", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINFDAQNTRTRDLERHFEPYGKIVNVRIRRNFAFIQYEAQEDATRALDATNSSKLMDKVISVEYAVK", "MKPVFCGNFEYDARESDLERLFRKYGKVERVDMKAGFAFVYMEDERDAEDAIRALDRFEYGRTGRRLRVEWTKNDRGGAGRSGGSRRSSSGLRPSKTLFVINF

In [5]:
prot.RGMeasureStep=50
prot.NSteps=tmp
#tmp = prot.NSteps
prot.EquilibrationTime=1000
#prot.NSteps=1000
#clusters = computeClusters2(prot; Cutoff=15.0);

LoadError: UndefVarError: tmp not defined

In [9]:
@time clusters= LammpsAnalysis.computeClusters(prot, Cutoff=8)
;

Function computeClusters does only detect charge contacts for clusters.
Step: Int32[451] of 4402
Step: Int32[951] of 4402
Step: Int32[1451] of 4402
Step: Int32[1951] of 4402
Step: Int32[2451] of 4402
Step: Int32[2951] of 4402
Step: Int32[3451] of 4402
Step: Int32[3951] of 4402
 15.357897 seconds (115.15 M allocations: 3.802 GiB, 5.26% gc time, 11.51% compilation time)


In [10]:
length.(clusters)
argmax.([length.(C) for C in clusters])
#argmax.(length.(clusters))

89-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [ ]:
function computeClusterCOMs(Sim::LammpsAnalysis.SimData{T,I2}, Clusters::Vector{Vector{Vector{I}}} ) where {I<:Integer, I2<:Integer, T<:Real}

    Coms= Vector{Matrix{T}}()
    for (step,C) in enumerate(Clusters)
        tmp = zeros(length(C),3 )
        for (CID,Cluster) in enumerate(C)
            mass=0
            for id in Cluster
                tmp[CID,:].+= Sim.COM[id,:,step]*Sim.ChainMasses[CID]
                mass+= Sim.ChainMasses[CID]
            end
            tmp[CID,:]./=mass
        end
        push!(Coms, tmp)
    end
    return Coms
end

computeClusterCOMs (generic function with 1 method)

In [11]:
bla = computeClusterCOMs(prot, clusters)

LoadError: UndefVarError: computeClusterCOMs not defined

In [ ]:
size(bla[1])

(27, 3)

In [12]:
function computeSlabHistogram(Sim::LammpsAnalysis.SimData,Clusters::Vector{Vector{Vector{I}}} ) where {I<:Integer}

    Clust_Coms = LammpsAnalysis.computeClusterCOMs(Sim, Clusters)
    LClustID = argmax.([length.(C) for C in clusters]) ### returns index for clusters with most proteins inside
    AxisCOM = [Clust_Coms[i][id, Sim.SlabAxis] for (i,id) in enumerate(LClustID)]

    if Sim.SlabAxis==1
        SlabCoord = Sim.x
    elseif Sim.SlabAxis==2
        SlabCoord = Sim.y  
    elseif Sim.SlabAxis == 3
        SlabCoord = Sim.z 
    else 
        ArgumentError("SlabAxis is not properly specified.")
    end


    NHists = 1 + 2 + Sim.NAtomTypes # one normal, one for positive chage, one for negative charge and one for each type

    ### array with 1:N steps for -boxwidth:boxwitdh in the direction of the slab
    Sim.SlabHistogramSeries = OffsetArray(zeros(eltype(Sim.x), Int32(ceil((Sim.BoxSize[Sim.SlabAxis,2]-Sim.BoxSize[Sim.SlabAxis,1])/Sim.Resolution)) , Int32(Sim.NSteps), NHists), Int32(ceil(Sim.BoxSize[Sim.SlabAxis,1]/Sim.Resolution))+1:Int32(ceil(Sim.BoxSize[Sim.SlabAxis,2]/Sim.Resolution)) , 1:Sim.NSteps, 1:NHists)

    AllAxis = [1,2,3]
    deleteat!(AllAxis, Sim.SlabAxis)

    #AxisCOM= [ sum(SlabCoord[:, step].*Sim.Masses)/(sum(Sim.Masses))  for step in 1:Sim.NSteps]


    ### convert from dalton/AA^3 to kg/L=g/mL  divided by Volume element per bin
    volume =((Sim.BoxSize[AllAxis[1],2]-Sim.BoxSize[AllAxis[1],1])*(Sim.BoxSize[AllAxis[2],2]-Sim.BoxSize[AllAxis[2],1]))
    conversion = 1.66053906660/volume/Sim.Resolution
    println("Conversion $(conversion), Volume: $(volume)")
    lowestind = Int32(ceil(Sim.BoxSize[Sim.SlabAxis,1]/Sim.Resolution))+1
    highestind = Int32(ceil(Sim.BoxSize[Sim.SlabAxis,2]/Sim.Resolution)) 
    printstyled("Subtract COM and add wrap afterwards\n"; color=:red)

    for (j,step) in enumerate(Sim.EquilibrationTime:Sim.RGMeasureStep:Sim.NSteps)### 1:NSteps
        for atom in 1:Sim.NAtoms ### 1:Atoms
            ### so far dont use particles that arent in the box
            ind = Int32(ceil((SlabCoord[atom,step]-AxisCOM[j])/Sim.Resolution))

            #ind = ind >= lowestind && ind<= highestind ?  ind : continue
            if ind < lowestind 
               # println("pre a: $ind")
                ind = highestind - (lowestind-ind)
                #println(ind)
            elseif  ind> highestind
                ind =lowestind + (ind-highestind)
            end

            Sim.SlabHistogramSeries[ind , step,1]+= Sim.Masses[atom]
            if Sim.Charges[atom] > 0
                Sim.SlabHistogramSeries[ind , step,2]+= Sim.Masses[atom]
            elseif Sim.Charges[atom] < 0
                Sim.SlabHistogramSeries[ind , step,3]+= Sim.Masses[atom]
            end
            Sim.SlabHistogramSeries[ind, step, Sim.IDs[atom]+3]+= Sim.Masses[atom] ### lowest ID is 1

        end
        for ind in axes(Sim.SlabHistogramSeries,1)
            for type in axes(Sim.SlabHistogramSeries,3)
                Sim.SlabHistogramSeries[ind, step,type] *= conversion
            end
        end 
    end
end

computeSlabHistogram (generic function with 1 method)

In [13]:
computeSlabHistogram(prot, clusters)

Conversion 4.555662734156378e-6, Volume: 72900.0
Subtract COM and add wrap afterwards
